In [1]:
import os
import pdb
import torch
import pandas
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
import torch.nn.functional as F

In [2]:
dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: CiteSeer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Has isolated nodes: True
Has self-loops: False
Is undirected: True


In [3]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        
        self.conv1 = GATConv(dataset.num_features, hidden_channels, dropout=0.6)
        self.conv2 = GATConv(hidden_channels, dataset.num_classes, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
    acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
    return acc

for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GAT(
  (conv1): GATConv(3703, 8, heads=1)
  (conv2): GATConv(8, 6, heads=1)
)
Epoch: 001, Loss: 1.7911, Val: 0.2260, Test: 0.2190
Epoch: 002, Loss: 1.7893, Val: 0.3000, Test: 0.2720
Epoch: 003, Loss: 1.7884, Val: 0.3860, Test: 0.3930
Epoch: 004, Loss: 1.7835, Val: 0.4740, Test: 0.4890
Epoch: 005, Loss: 1.7852, Val: 0.6060, Test: 0.5870
Epoch: 006, Loss: 1.7804, Val: 0.5480, Test: 0.6000
Epoch: 007, Loss: 1.7864, Val: 0.4540, Test: 0.4810
Epoch: 008, Loss: 1.7821, Val: 0.4880, Test: 0.5170
Epoch: 009, Loss: 1.7799, Val: 0.5760, Test: 0.5910
Epoch: 010, Loss: 1.7741, Val: 0.6560, Test: 0.6630
Epoch: 011, Loss: 1.7740, Val: 0.6320, Test: 0.6380
Epoch: 012, Loss: 1.7799, Val: 0.6140, Test: 0.5990
Epoch: 013, Loss: 1.7787, Val: 0.6040, Test: 0.5870
Epoch: 014, Loss: 1.7696, Val: 0.5920, Test: 0.5770
Epoch: 015, Loss: 1.7653, Val: 0.6140, Test: 0.5970
Epoch: 016, Loss: 1.7649, Val: 0.6300, Test: 0.6330
Epoch: 017, Loss: 1.7657, Val: 0.6580, Test: 0.6600
Epoch: 018, Loss: 1.7568, Val: 0.6680,

In [5]:
test_acc = test(data.val_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7060
